# Car speed over time

In this part of the project the development of the speed of the cars will be investigated to answer the question: How much quicker did the cars get over time?

For this the following points will be investigated using the data that was prepared in the previous part:
- How much quicker did the fastest cars get?
- How much quicker did the slowest cars get?
- How much quicker did the cars on average get?

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("F1") \
    .getOrCreate()

25/04/28 09:07:41 WARN Utils: Your hostname, bdlc-004 resolves to a loopback address: 127.0.1.1; using 10.176.129.6 instead (on interface ens192)
25/04/28 09:07:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/28 09:07:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
!hdfs dfs -ls /f1/data/cleaned_*.parquet

Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-04-27 17:08 /f1/data/cleaned_circuits.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup       6904 2025-04-27 17:08 /f1/data/cleaned_circuits.parquet/part-00000-99e32ab9-9e0f-42b2-a6d3-42d4ef687d18-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-04-27 17:30 /f1/data/cleaned_constructor_results.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup      80657 2025-04-27 17:30 /f1/data/cleaned_constructor_results.parquet/part-00000-23998559-62d1-4e57-baa4-d4d192fefdfc-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-04-27 17:33 /f1/data/cleaned_constructor_standings.parquet/_SUCCESS
-rw-r--r--   2 cluster supergroup     110395 2025-04-27 17:33 /f1/data/cleaned_constructor_standings.parquet/part-00000-a866fd49-8cdf-4567-95b0-47dec9a9f110-c000.snappy.parquet
Found 2 items
-rw-r--r--   2 cluster supergroup          0 2025-04-27 17:34 /f1/data/cleaned_constructo

As a first step all the necessary datasets will be loaded and how they look like.

In [9]:
circuits_df = spark.read.parquet(f"/f1/data/cleaned_circuits.parquet")
lap_times_df = spark.read.parquet(f"/f1/data/cleaned_lap_times.parquet")
races_df = spark.read.parquet(f"/f1/data/cleaned_races.parquet")
# maybe use drivers
results_df = spark.read.parquet(f"/f1/data/cleaned_results.parquet")
# seasons_df = spark.read.parquet(f"/f1/data/cleaned_seasons.parquet")

In [10]:
circuits_df.show(1)
lap_times_df.show(1)
races_df.show(1)
results_df.show(1)
# seasons_df.show(1)

+---------+-----------+--------------------+---------+---------+--------+-------+
|circuitId| circuitRef|                name| location|  country|     lat|    lng|
+---------+-----------+--------------------+---------+---------+--------+-------+
|        1|albert_park|Albert Park Grand...|Melbourne|Australia|-37.8497|144.968|
+---------+-----------+--------------------+---------+---------+--------+-------+
only showing top 1 row

+------+--------+---+--------+--------+------------+
|raceId|driverId|lap|position|    time|milliseconds|
+------+--------+---+--------+--------+------------+
|   990|     840| 27|      16|1:38.414|       98414|
+------+--------+---+--------+--------+------------+
only showing top 1 row

+------+----+-----+---------+--------------------+----------+--------+--------+--------+--------+--------+--------+--------+----------+----------+-----------+-----------+
|raceId|year|round|circuitId|                name|      date|    time|fp1_date|fp1_time|fp2_date|fp2_time|

In [32]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

fastest_laps = lap_times_df.groupBy("raceId").agg(F.min("milliseconds").alias("fastest_lap"))
races_with_year = races_df.select("raceId", "year")
fastest_laps_with_year = fastest_laps.join(races_with_year, "raceId")

speed_development = fastest_laps_with_year.join(
    races_df, fastest_laps_with_year.raceId == races_df.raceId
).groupBy("circuitId", fastest_laps_with_year.year).agg(F.min("fastest_lap").alias("min_fastest_lap"))
speed_development.show(5)


Py4JJavaError: An error occurred while calling o639.showString.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:77)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:481)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.comma

# Investigate one specific circuit over the years - Interlagos, Brazil
This has been altered last time in 1990 so interesting to see this
In a first step only one circuit will be investigated